The source of the algorithm is inside several .py files. Accessing through bovw's run method we can configure the execution in order to iterate over the desired parameters. If a parameter has exactly one value, only that will be use. If it has tree, a np.linspace will be created with them.

In [ ]:
from bovw import run

We use the following default method to run the experiments.

In [ ]:
from argparse import Namespace
import numpy as np
from matplotlib import pyplot as plt

def run_experiment(method=('sift',), n_features=(300,), step_size=(16,), n_clusters=(128,), n_neighbors=(5,), 
                   distance=('euclidean',), confusion_matrix=False):
    args = Namespace(train_path='../data/MIT_split/train',
                     test_path='../data/MIT_split/test',
                     method=method,
                     n_features=n_features,
                     step_size=step_size,
                     n_clusters=n_clusters,
                     n_neighbors=n_neighbors,
                     distance=distance,
                     confusion_matrix=confusion_matrix)
    return run(args)

#### Test different amounts of local features using SIFT
If we don't have enough features the results will be bad but if we have too many the computational time will increase too much. We should find a compromise between both.

In [ ]:
results = run_experiment(n_features=[100, 1000, 10])
plt.plot(results['n_features'], results['accuracy'])
plt.xlabel('n_features')
plt.ylabel('accuracy')
plt.show()

The accuracy from 500 features onwards is more or less constant. So a value between 500 and 600 should work properly.

#### SIFT vs DenseSIFT
Dense SIFT consists in running SIFT on a dense gird of locations at a fixed scale and orientation. This kind of descriptors have proven to yield better results for image classification tasks.

In [ ]:
results = run_experiment(method=['sift', 'dense_sift'])
x = np.arange(2)
plt.bar(x, results['accuracy'])
plt.xticks(x, results['method'])
plt.xlabel('method')
plt.ylabel('accuracy')
plt.show()

In the graph above, we can crearly observe how `dense_sift` outperforms `sift`.

#### Test different amounts of codebook sizes k

In [ ]:
results = run_experiment(method=['sift', 'dense_sift'], n_clusters=[80, 400, 25])
results_sift = results.loc[results['method'] == 'sift']
results_dense_sift = results.loc[results['method'] == 'dense_sift']
plt.plot(results_sift['n_clusters'], results_sift['accuracy'], label='sift')
plt.plot(results_dense_sift['n_clusters'], results_dense_sift['accuracy'], label='dense_sift')
plt.xlabel('n_clusters')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.show()

#### Test different values of k for the k-nn classifier

In [ ]:
results = run_experiment(method=['sift', 'dense_sift'], n_neighbors=[3, 21, 10])
results_sift = results.loc[results['method'] == 'sift']
results_dense_sift = results.loc[results['method'] == 'dense_sift']
plt.plot(results_sift['n_neighbors'], results_sift['accuracy'], label='sift')
plt.plot(results_dense_sift['n_neighbors'], results_dense_sift['accuracy'], label='dense_sift')
plt.xlabel('n_neighbors')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.show()

The number of neighbors must be at least 10 to provide good enough results.

#### Test different distances for the k-nn classifier

In [ ]:
results = run_experiment(method=['sift', 'dense_sift'], distance=['euclidean', 'manhattan', 'chebyshev'])
results_sift = results.loc[results['method'] == 'sift']
results_dense_sift = results.loc[results['method'] == 'dense_sift']
x = np.arange(3)
plt.bar(x-0.1, results_sift['accuracy'], width=0.2, align='center', label='sift')
plt.bar(x+0.1, results_dense_sift['accuracy'], width=0.2, align='center', label='dense_sift')
plt.xticks(x, ['euclidean', 'manhattan', 'chebyshev'])
plt.xlabel('distance')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.show()

The euclidean distance is clearly the best performing one.

#### Confusion matrix
Finally, we plot the confusion matrix of the best combination of hyperparameters.

In [ ]:
run_experiment(method=['dense_sift'], n_clusters=[250], n_neighbors=[11], distance=['euclidean'], confusion_matrix=True)